In [ ]:
import keras
import tensorflow as tf
from keras import layers
import pandas as pd
import numpy as np
from tensorflow.keras.losses import binary_crossentropy
import os
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from scipy import stats
import matplotlib.pyplot as plt
import time
from keras.models import Sequential
import copy
import random
from operator import attrgetter
from tensorflow.keras import initializers


# Autoencoder Function - N Dimensions

In [ ]:
nit = 112
epochs=10000
batch=64

data = pd.read_csv("Respostas.csv")


callback = keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-8,patience=50,
                                         verbose=0,mode="min",
                                        restore_best_weights=True)


Qmat = pd.read_csv("Qmat.csv")
Qmat = Qmat.T
Qmat = tf.constant(Qmat, dtype=tf.float32)


In [ ]:

def a_reg(a):
    a = a * K.cast(K.greater_equal(a, 0), K.floatx())
    a = a + K.cast(K.equal(a, 0), K.floatx())/10
    
    nrows = a.get_shape()[0]
    new_weights = []
    
    aux = Qmat.numpy()
        
    for j in range(nrows):

        index = np.where(aux[j,:] != 0)
        index = np.reshape(index,(np.sum(aux[j,:] != 0),))

        column = a[j,:]


        values = tf.gather(column, index)
        t_values = tf.exp(tf.math.log(values) - 1/nit * tf.reduce_sum(tf.math.log(values)))
        index = index.reshape(-1, 1)

        new_column = tf.tensor_scatter_nd_update(column, index, t_values)
        new_weights.append(new_column)
    new_weights = tf.stack(new_weights, axis=0)
    new_weights = Qmat * new_weights
    
    return new_weights

def b_reg(b):
   return b-K.sum(b)/nit;  


def AEIRT(n_layers, vector_neurons, n_items, data, epochs, batch, restriction, n_latent):

    inputs = Input(shape=(n_items,))
    
    if(n_layers == 0):
        encoded = Dense(units=n_latent, activation='linear')(inputs)
    else:
        x1 = Dense(vector_neurons[0], activation = 'relu')(inputs)
        for i in range(n_layers-1):
            x1 = Dense(vector_neurons[i+1], activation = 'relu')(x1)
        encoded = Dense(units=n_latent, activation='linear')(x1)    
    
    decoder_inputs = Input(shape=(n_latent,))
    
    # Op 1
    if restriction == 'Product':
        outputs = Dense(n_items, activation='sigmoid', 
                    bias_constraint=b_reg,
                    kernel_constraint=a_reg,
#                    kernel_constraint=CustomConstraint(Q_mat = Q_mat, nit = n_items),
                    kernel_initializer=initializers.Ones(),
                    bias_initializer=initializers.Zeros())(decoder_inputs)
        encoder = Model(inputs, encoded, name='encoder')
        decoder = Model(decoder_inputs, outputs, name='decoder')


    outputs = decoder(encoded)
    vae = Model(inputs, outputs, name='vae')

    vae.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    history = vae.fit(data,data,
                epochs=epochs,
                shuffle = True, 
                batch_size=batch,
                validation_split = 0.2,
                verbose = 0,callbacks=[callback])
    
    latent = encoder.predict(data, verbose = 0).flatten()
    
    return history, decoder, latent


# Genetic Algorithm Function

In [ ]:
# Settings

POPULATION_SIZE = 5
MUTATION_RATE = 0.40
GENERATIONS = 5
ELITISM = 0.80
NUMBER_CHILDS = 3 


In [ ]:
class Individual:
    def __init__(self, n_layers, vector_neurons):
        self.n_layers = n_layers
        self.vector_neurons = vector_neurons
        self.fitness = None
        self.discr = None
        self.diff = None
        self.latent = None
        
    def evaluate_fitness(self, data, epochs, batch, restriction, n_latent):               
        history, decoder, latent = AEIRT(self.n_layers, self.vector_neurons, nit, data, epochs, batch, restriction, n_latent)
        self.fitness = history.history['val_loss'][-1]
        self.discr = decoder.get_weights()[0]
        self.diff = decoder.get_weights()[1]
        self.latent = latent
        
def generate_individual():
    n_layers = random.randint(0, 5)
    vector_neurons = []
    for i in range(n_layers):
        aux = random.randint(2, nit*2)
        aux = aux // 5 * 5 
        aux += 1
        vector_neurons.append(aux)
    return Individual(n_layers, vector_neurons)

def crossover(parent1, parent2):
      
    child1 = Individual(parent1.n_layers, parent1.vector_neurons)
    child2 = Individual(parent2.n_layers, parent2.vector_neurons)
        
# It is randomized the changes in the vector of neurons
    for i in range(child1.n_layers):
        if parent1.n_layers <= i:
            child1.vector_neurons[i] = parent2.vector_neurons[i]
        elif parent2.n_layers <= i:
            child1.vector_neurons[i] = parent1.vector_neurons[i]
        else:
            if random.random() < 0.5:  
                child1.vector_neurons[i] = parent2.vector_neurons[i]
            else:
                child1.vector_neurons[i] = parent1.vector_neurons[i]
                
    for i in range(child2.n_layers):
        if parent1.n_layers <= i:
            child2.vector_neurons[i] = parent2.vector_neurons[i]
        elif parent2.n_layers <= i:
            child2.vector_neurons[i] = parent1.vector_neurons[i]
        else:
            if random.random() < 0.5:  
                child2.vector_neurons[i] = parent2.vector_neurons[i]
            else:
                child2.vector_neurons[i] = parent1.vector_neurons[i]
                
    return child1, child2

def mutate(individual):
    if random.random() < MUTATION_RATE:
        individual.n_layers += random.randint(-1, 1)
        if individual.n_layers < 0:
            individual.n_layers = 0
        if individual.n_layers > 5:
            individual.n_layers = 5
                
        if individual.n_layers > len(individual.vector_neurons):
            aux = random.randint(2, nit*2)
            aux = aux // 5 * 5 
            individual.vector_neurons.append(aux)
        if individual.n_layers < len(individual.vector_neurons):
            del individual.vector_neurons[-1]
                
    for i in range(len(individual.vector_neurons)):
        if random.random() < 0.5:
            aux = random.randint(-10, 10)
            aux = aux // 5 * 5
            individual.vector_neurons[i] += aux
            if individual.vector_neurons[i] < 1:
                individual.vector_neurons[i] = 2
    return individual

def select_parents(population, k):
    tournament = random.sample(population, k)
    parent1 = max(tournament, key=lambda x: x.fitness)
    tournament.remove(parent1)
    parent2 = max(tournament, key=lambda x: x.fitness)  
    return parent1, parent2

def evolve(population, data, epochs, batch, restriction, n_latent):
    for i in range(GENERATIONS):
        print(f"Generation {i+1}")
        population.sort(key=attrgetter('fitness'))
        elite_count = int(ELITISM * len(population))
        new_population = population[:elite_count]
        for j in range(NUMBER_CHILDS):
            if random.random() < 0.6:
                parent1, parent2 = select_parents(population,POPULATION_SIZE)
                parent1 = copy.deepcopy(parent1)
                parent2 = copy.deepcopy(parent2)  
                child1, child2 = crossover(parent1, parent2)
            else:
                child1 = generate_individual()
                child2 = generate_individual()
            child1 = mutate(child1)
            child2 = mutate(child2)

            try:
                child1.evaluate_fitness(data, epochs, batch, restriction, n_latent)
            except:
                child1.fitness = 100


            try:
                child2.evaluate_fitness(data, epochs, batch, restriction, n_latent)
            except:
                child2.fitness = 100
                
            new_population.append(child1)
            new_population.append(child2)
        population = new_population
        population.sort(key=attrgetter('fitness'))
        population = population[:POPULATION_SIZE]     

#     Only if we want to check how the population is evolving
#        for i in range(POPULATION_SIZE):
#            print(f"Layers: {population[i].n_layers} -- {population[i].vector_neurons}")

    population.sort(key=attrgetter('fitness'))
    return population


# Product Restriction

In [ ]:
individuals = []
for i in range(POPULATION_SIZE):
    individual = generate_individual()
    individual.evaluate_fitness(data.to_numpy(), epochs, batch, 'Product', 3)
    individuals.append(individual)
#    print(f"Layers: {pops[i].n_layers} -- {pops[i].vector_neurons} -- Loss: ", pops[i].fitness)

In [ ]:
pops = evolve(population = individuals,data=data.to_numpy(),
              epochs=epochs, batch=batch, restriction = 'Product', n_latent = 3)

In [ ]:
for i in range(len(pops)):
    print(f"Layers: {pops[i].n_layers} -- {pops[i].vector_neurons} -- Loss: ", pops[i].fitness)

# Evaluate

In [ ]:
pd.DataFrame(pops[0].discr).T.to_csv('EnemDiscr.csv')
pd.DataFrame(pops[0].diff).to_csv('EnemDiff.csv')
pd.DataFrame(pops[0].latent).to_csv('EnemLatent.csv')